# NER

Spacy vs flair

In [2]:
from tika import parser
import string
from flair.data import Sentence
from flair.models import SequenceTagger
import re
import requests
import io
from bs4 import BeautifulSoup
import spacy

/home/genta/Documents/notebooks_cs/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In the next cell you have to comment in the part of the code that describes your situation. The first part is to load the PDF from the E-Periodica website without actually downloading it onto your machine, the second part is for the case where you already have the PDF on your machine.

### PDF
If you use PDFs from a Website or from your computer, run these cells

In [3]:
# If you don't want to save the PDF locally, comment in the next four lines and comment out the last one.
url = "https://www.e-periodica.ch/cntmng?pid=grs-001%3A1921%3A13%3A%3A298"
r = requests.get(url)
f = io.BytesIO(r.content)
parsed = parser.from_buffer(f)

# If you've downloaded the PDF onto your computer, comment in the following line:
#parsed = parser.from_file('data/ner_data/grs-001_1921_13__298_d.pdf')

The E-Periodica PDFs already have the OCR embedded into them, so all you need to do is extract the text and clean it up.

In [4]:
#pdf
contents = [x.strip() for x in parsed["content"].split("\n") if x != ""]
#remove the first page
article = contents[contents.index('https://www.e-periodica.ch/digbib/about3?lang=en')+1:]
article = " ".join(article)

article = re.sub("¬\s+", "", article)  # "bindestriche" will be removed, if they are followed by one or several whitespaces, those will be removed as well.
article = article.strip()  # remove all starting and trailing whitespaces
article = re.sub("\n", " ", article)  # replace newlines with spaces
article = re.sub("\. ", "\.\n", article)  # replace periods with newlines (for nicer printing)
article = re.sub(r'\s+', " ", article)  # replace all repeating whitespaces with only one whitespace
article = re.sub(r'\\', "", article)  # replace all double backslashes

#### Text Output
If you would like to now save this PDF text as a simple text file, run this cell:

In [8]:
output_filename = "data/ner_data/output/grs-001_1921_13__298_d.txt"
with open(output_filename,"w") as f:
    f.writelines(article)

### Text
If you have the input as a text file, run this cell:

In [9]:
#text
input_filepath = "data/ner_data/output/grs-001_1921_13__298_d.txt"
with open(input_filepath, "r") as f:
    article = f.read()

### XML
If you have the input as an XML file, run this cell:

In [17]:
#xml
input_filepath = "/home/genta/Documents/notebooks_cs/data/ocr_data/output/grs-001_1921_13__298_d_tei.xml"
with open(input_filepath, "r") as f:
    article = f.read()
soup = BeautifulSoup(article, features="xml")
pageText = soup.findAll(text=True)
article = " ".join(pageText)

/tmp/ipykernel_532672/3857886091.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  pageText = soup.findAll(text=True)


In [18]:
article

'GEWERKSCHAFTLICHE RUNDSCHAU 39 Lohnabbau. In den Mittelpunkt des Interesses ist der «Lohnabbau» gerückt. Auch er ist wie die Krise eine internationale Erscheinung. In Amerika, in England, ja sogar in den valutaschwachen Ländern Deutschland. Oesterreich, Tschechoslowakei. Italien usw.. überall stehen wir vor der gleichen Erscheinung. Der « Preisabbau» hatte kaum Zeit, sich anzumelden, machton schon die Industriellen die grössten Anstrengungen, die «hohen» Kriegslöhne auf ein «erträgliches Mass» zurückzudrücken. Leider ist die Situation diesem Vorhaben günstig, denn die Konkurrenz der arbeitslosen Reservearmee war noch nie so stark wie oben jetzt. Es lässt sich nachweisen, dass hauptsächlich in der Textilindustrie die Löhne gesunken sind, ohne dass die Oeffentlichkeit etwas davon merkte. In den Heimarbeitgebieten ist es nicht besser. Lohnreduktionen treten auf in der chemischen Industrie und in manchen Zweigen der Lebens- und Genussmittelindustrie. Unberührt davon sind bis jetzt nur die

### Image
If you have the input as an image file only, please check out the OCR notebook, save the text files and run the cells for the Text input.

## Flair
First, we'll show you how named entity tagging works with FlairNLP

In [19]:
# load tagger, this might take a while
tagger = SequenceTagger.load("flair/ner-german-large")

# predict on the article
sentence = Sentence(article)
tagger.predict(sentence)

2025-03-20 16:08:55,752 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


In [20]:
people = []
places = []
organisations = []

for entity in sentence.get_spans('ner'):
    if entity.tag == "PER": #people
        name = entity.text.translate(str.maketrans('', '', string.punctuation)) #remove possible ocr mistakes
        if len(name) >= 3: # names are usually not shorter
            people.append(entity.text)
    elif entity.tag == "LOC": #places
        place = entity.text.translate(str.maketrans('', '', string.punctuation)) #remove possible ocr mistakes
        if len(place) >= 3: # place are usually not shorter
            places.append(entity.text)
    elif entity.tag == "ORG": #only organisations
        org = entity.text.translate(str.maketrans('', '', string.punctuation)) #remove possible ocr mistakes
        if len(org) >= 3:
            organisations.append(entity.text)

In [21]:
people

[]

In [22]:
places

['Amerika',
 'England',
 'Deutschland',
 'Oesterreich',
 'Tschechoslowakei',
 'Italien',
 'Schweiz',
 'Bern',
 'Schweiz',
 'Frankreich',
 'Italien',
 'Deutschland',
 'Holland',
 'Schweiz',
 'Deutschland',
 'Schweiz',
 'Nordfrankreich',
 'Belgien',
 'Genf',
 'Europas',
 'Europa',
 'Deutschland',
 'Deutschlands',
 'Frankreich',
 'Belgien',
 'Deutschland',
 'England',
 'Amsterdam',
 'Deutschland',
 'Deutschland',
 'Deutschlands']

In [23]:
organisations

['Metall',
 'und Uhrcnarbeiter-Verband',
 'V. S. K.',
 'Internationale Gewerkschaftsbund zum Wiederaufbau',
 'Internationalen Gewerkschafts bundes',
 'Internationale Gewerkschaftsbund',
 'Internationalen Gewerkschaftsbundes']

But this is only People / Places / Organisations. What if you also want to extract dates and numbers (cardinal) and times? For that you need the "ontonotes" model. FlairNLP only has this model trained on english content, but as you'll see, it will work surprisingly well even on German text.

In [24]:
tagger_onto = SequenceTagger.load("flair/ner-english-ontonotes-large")

2025-03-20 16:10:42,554 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [25]:
sent_test = Sentence("Dem will eine Datenschutztagung an der ETH Zürich dienen, die von einer Gruppe Gewerkschaftern organisiert und vom SGB und seinen Verbänden unterstützt wird. Sie findet Samstag, den 31. März 1984 ab 9.15 Uhr, ganztägig statt.")
tagger_onto.predict(sent_test)

In [26]:
sent_test.get_spans('ner') #not bad, huh?

[Span[5:8]: "der ETH Zürich" → ORG (1.0000),
 Span[18:19]: "SGB" → ORG (1.0000),
 Span[27:33]: "Samstag, den 31. März 1984" → DATE (1.0000),
 Span[34:36]: "9.15 Uhr" → TIME (0.8940)]

## Spacy
Now we'll do the same thing, but with Spacy. The results for our toy examples are identical between spacy and flair, but keep in mind that spacy is significantly faster.

### German NER
First, just as with FlairNLP, we show the results for regular German entity tagging.

In [30]:
nlp_de = spacy.load("de_core_news_lg")
doc_de = nlp_de(article)

In [31]:
nlp_de.get_pipe('ner').labels

('LOC', 'MISC', 'ORG', 'PER')

Unlike FlairNLP though, Spacy has a really nice visualization capability. 

In [32]:
spacy.displacy.serve(doc_de, style="ent")

/home/genta/Documents/notebooks_cs/.venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [33]:
# Find named entities, phrases and concepts
for entity in doc_de.ents:
    print(entity.text, entity.label_)

GEWERKSCHAFTLICHE MISC
Lohnabbau ORG
Amerika LOC
England LOC
valutaschwachen Ländern LOC
Deutschland LOC
Oesterreich LOC
Tschechoslowakei LOC
Italien LOC
Kriegslöhne PER
Reservearmee ORG
Heimarbeitgebieten LOC
Lohnreduktionen PER
Zeit MISC
Lohnabbau ORG
Sic PER
Exportindustric LOC
Baulust MISC
Preisabbaues PER
Löhnen MISC
deutschen MISC
österreichischen MISC
Schweiz LOC
Metallarbeitern LOC
Bern LOC
V. S. K. PER
deutschen MISC
Gewiss PER
deutsche MISC
Schweizer MISC
Mark im Jahr MISC
Schweizer MISC
Ernährungsamtes PER
der Schweiz LOC
Frankreich LOC
Rp MISC
Italien LOC
Rp MISC
Deutschland LOC
Rp MISC
Holland LOC
Rp MISC
Schweiz LOC
Rp MISC
Tn MISC
Deutschland LOC
Schweizer MISC
Lohnabbau ORG
Bund LOC
Kommission ORG
Bundeshaus LOC
hinblicken LOC
polnische MISC
GEWERKSCHAFTLICHE MISC
RUNDSCHAU MISC
Allgemeininteressen LOC
Intcressencliquen MISC
Schweiz LOC
schweizerische MISC
schweizerische MISC
Landes LOC
Verhältnisesn PER
Internationale Gewerkschaftsbund ORG
Nordfrankreich LOC
Belgien LO

But what if we have some text with lots of time and dates?

In [34]:
doc_2 = nlp_de("Dem will eine Datenschutztagung an der ETH Zürich dienen, die von einer Gruppe Gewerkschaftern organisiert und vom SGB und seinen Verbänden unterstützt wird. Sie findet Samstag, den 31. März 1984 ab 9.15 Uhr, ganztägig statt.")
# Find named entities, phrases and concepts
for entity in doc_2.ents:
    print(entity.text, entity.label_)

ETH Zürich ORG
Gewerkschaftern ORG
SGB MISC


### English Onto NER
In that case we need to step it up once again. Using a model trained on the ontonotes tags once again, but once again only the english ones, we get many more labels to use.

In [5]:
nlp = spacy.load("en_core_web_trf")
doc = nlp(article)

In [6]:
nlp.get_pipe('ner').labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [7]:
spacy.displacy.serve(doc, style="ent")

/home/genta/Documents/notebooks_cs/.venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [8]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

39 CARDINAL
Amerika LOC
England GPE
Ländern Deutschland GPE
Oesterreich GPE
Tschechoslowakei GPE
1. Mai DATE
10 % PERCENT
Maitage ORG
Exportindustric ORG
seit Monaten DATE
1921 DATE
1-9J4 DATE
150 bis 200 % PERCENT
150 % PERCENT
100 % PERCENT
150 % PERCENT
100 % PERCENT
Bern GPE
124 % PERCENT
89 % PERCENT
am 1. April 1921 DATE
1914 DATE
130,64 % PERCENT
Vergleiche PERSON
Gewiss PERSON
Valuta LOC
60 Mark MONEY
1500 Mark QUANTITY
Monat DATE
18.000 Mark MONEY
im Jahr DATE
Schweizer GPE
6 CARDINAL
Dagegen PERSON
Brot PERSON
Frankreich GPE
53 Rp MONEY
Italien GPE
46 Rp MONEY
Deutschland GPE
24 Rp MONEY
Holland GPE
63 Rp MONEY
Schweiz GPE
76 Rp MONEY
Im allgemeinen PERSON
Deutschland GPE
etwa 800 % PERCENT
Arm in Arm FAC
Zerealien FAC
Eier ORG
Vieh ORG
Bund ORG
Loch LOC
Bundeskasse FAC
Volkes ORG
40 CARDINAL
vier Millionen CARDINAL
Nordfrankreich GPE
Belgien GPE
Genf GPE
am 17. Februar 1921 DATE
Das Bureau des Internationalen Gewerkschafts bundes ORG
14. und 15. März DATE
u. a. PERSON
Das Bu

But does it work on our German text?

In [9]:
doc_2 = nlp("Dem will eine Datenschutztagung an der ETH Zürich dienen, die von einer Gruppe Gewerkschaftern organisiert und vom SGB und seinen Verbänden unterstützt wird. Sie findet Samstag, den 31. März 1984 ab 9.15 Uhr, ganztägig statt.")
# Find named entities, phrases and concepts
for entity in doc_2.ents:
    print(entity.text, entity.label_)

ETH Zürich ORG
SGB ORG
Samstag DATE
den 31. März 1984 DATE
9.15 Uhr TIME


It sure does! Looks great, despite the fact that this model was only trained on English data. That implies that there's some kind of generality to the rules it is learning for entity tagging.